# Water Security

### Downloading all data

In [1]:
# Make sure you've run aws configure before hand
!aws s3 sync s3://s3groupegypt data

## Preprocessing

In [2]:
import pandas as pd
import numpy as np
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
import geopandas
import geoplot
import h5py
import warnings
warnings.filterwarnings('ignore')

# Merge 0.5x0.5 degree cells into 1x1 degree cells.
# We overlay 1x1 degree cells on the world and intersect with co-ordinates
# and aggregate using average.
# This is pretty generic and infact works with any x degree cell.
def overlay_grid_cells(degrees=1):
    grid = []
    for lat in np.arange(-180, 180, degrees):
        for lon in np.arange(-360, 360, degrees):
            grid.append(box(lat, lon, lat+degrees, lon+degrees))
    return grid

def geodataframe_merge_adjacent_cells(df, degrees=1, aggfunc='mean'):
    columns = df.columns[:-1]
    grid = pd.DataFrame(data={'id': np.arange(0, 360*720)})
    crs = {'init': 'epsg:4326'}
    gf = geopandas.GeoDataFrame(grid, crs=crs, geometry=overlay_grid_cells(degrees=degrees))
    gf_intersect = geopandas.sjoin(gf, df, how="inner", op='intersects')
    b = gf_intersect.dissolve(by='id', aggfunc = 'mean')
    return geopandas.GeoDataFrame(b[columns], geometry=b.geometry.centroid)

# Converts dataframe to geodataframe
# Dataframe must contain lon/lat fields!
def dataframe_to_geodataframe(df):
    crs = {'init': 'epsg:4326'}
    geo = [Point(row['lon'], row['lat']) for (_, row) in df[['lon', 'lat']].iterrows()]
    gdf = geopandas.GeoDataFrame(df, crs=crs, geometry=geo)
    return gdf

frames = []

### Loading SSP2 dataset

Here we load the dataset and visualise the information with a head()/google map

In [3]:
# This dataset has population estimates every decade. We linearly impute 
# the intermediate years.
def linear_impute(df, prefix):
    df[["%c2_%s" % (prefix, i) for i in np.arange(1980, 2017) if i % 10]] = np.nan
    df_t = df[["%c2_%s" % (prefix, i) for i in np.arange(1980, 2017)]].transpose()
    df_it = df_t.interpolate(method='linear', direction = 'forward').transpose()
    df_it[["lon", "lat"]] = df[["px", "py"]]
    return df_it

In [4]:
# Read, Impute, GeoDataFrame, Center on grid
population = pd.read_csv("data/SSP2/pop_ssp2.csv")
population = linear_impute(population, prefix="p")
population = dataframe_to_geodataframe(population)
population = geodataframe_merge_adjacent_cells(population)

gdp = pd.read_csv("data/SSP2/gdp_ssp2.csv")
gdp = linear_impute(gdp, prefix="g")
gdp = dataframe_to_geodataframe(gdp)
gdp = geodataframe_merge_adjacent_cells(gdp)

frames.append(population)
frames.append(gdp)

In [5]:
gdp.head()

,g2_1980,g2_1981,g2_1982,g2_1983,g2_1984,g2_1985,g2_1986,g2_1987,g2_1988,g2_1989,...,g2_2010,g2_2011,g2_2012,g2_2013,g2_2014,g2_2015,g2_2016,lon,lat,geometry
id,,,,,,,,,,,,,,,,,,,,,
340,0.011155,0.011242,0.011329,0.011416,0.011503,0.011590,0.011677,0.011764,0.011851,0.011938,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,-179.861740,-19.004300,POINT (-179.50000 -19.50000)
341,0.013710,0.013812,0.013914,0.014015,0.014117,0.014219,0.014321,0.014422,0.014524,0.014626,...,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052,-179.528190,-18.632700,POINT (-179.50000 -18.50000)
342,0.013705,0.013801,0.013898,0.013994,0.014090,0.014186,0.014283,0.014379,0.014475,0.014572,...,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,-179.298957,-17.479223,POINT (-179.50000 -17.50000)
343,0.013764,0.013868,0.013972,0.014076,0.014180,0.014284,0.014388,0.014492,0.014596,0.014700,...,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,-179.673520,-16.418920,POINT (-179.50000 -16.50000)
344,0.010590,0.010685,0.010780,0.010876,0.010971,0.011066,0.011162,0.011257,0.011352,0.011448,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-179.702275,-15.829730,POINT (-179.50000 -15.50000)


In [6]:
population.head()

,p2_1980,p2_1981,p2_1982,p2_1983,p2_1984,p2_1985,p2_1986,p2_1987,p2_1988,p2_1989,...,p2_2010,p2_2011,p2_2012,p2_2013,p2_2014,p2_2015,p2_2016,lon,lat,geometry
id,,,,,,,,,,,,,,,,,,,,,
340,0.003656,0.003699,0.003742,0.003785,0.003828,0.003871,0.003914,0.003957,0.003999,0.004042,...,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,-179.861740,-19.004300,POINT (-179.50000 -19.50000)
341,0.004463,0.004515,0.004566,0.004617,0.004669,0.004720,0.004771,0.004823,0.004874,0.004925,...,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,-179.528190,-18.632700,POINT (-179.50000 -18.50000)
342,0.004357,0.004408,0.004458,0.004509,0.004560,0.004610,0.004661,0.004712,0.004762,0.004813,...,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,-179.298957,-17.479223,POINT (-179.50000 -17.50000)
343,0.004480,0.004532,0.004584,0.004636,0.004688,0.004741,0.004793,0.004845,0.004897,0.004949,...,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,-179.673520,-16.418920,POINT (-179.50000 -16.50000)
344,0.003656,0.003699,0.003742,0.003785,0.003828,0.003871,0.003914,0.003957,0.003999,0.004042,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-179.702275,-15.829730,POINT (-179.50000 -15.50000)


### Loading Water GAP dataset

Here we load the dataset and visualise the information with a head()/google map

### Loading GLDAS dataset

Here we load the dataset and visualise the information with a head()/google map

### Merging the dataframe

Here we load the dataset and visualise the information with a head()/google map

In [20]:
def geodataframe_spatial_concat(frames):
    main = frames[0]
    for frame in frames[1:]:
        main = master_df.merge(frame)
    return main
master_df = geodataframe_spatial_concat(frames)
master_df.head()

,p2_1980,p2_1981,p2_1982,p2_1983,p2_1984,p2_1985,p2_1986,p2_1987,p2_1988,p2_1989,...,g2_2007,g2_2008,g2_2009,g2_2010,g2_2011,g2_2012,g2_2013,g2_2014,g2_2015,g2_2016
0,0.003656,0.003699,0.003742,0.003785,0.003828,0.003871,0.003914,0.003957,0.003999,0.004042,...,0.004614,0.003077,0.001540,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
1,0.004463,0.004515,0.004566,0.004617,0.004669,0.004720,0.004771,0.004823,0.004874,0.004925,...,0.005875,0.003934,0.001993,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052
2,0.004357,0.004408,0.004458,0.004509,0.004560,0.004610,0.004661,0.004712,0.004762,0.004813,...,0.005821,0.003895,0.001969,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043
3,0.004480,0.004532,0.004584,0.004636,0.004688,0.004741,0.004793,0.004845,0.004897,0.004949,...,0.005907,0.003955,0.002002,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050
4,0.003656,0.003699,0.003742,0.003785,0.003828,0.003871,0.003914,0.003957,0.003999,0.004042,...,0.004429,0.002953,0.001476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
master_df

,p2_1980,p2_1981,p2_1982,p2_1983,p2_1984,p2_1985,p2_1986,p2_1987,p2_1988,p2_1989,...,g2_2007,g2_2008,g2_2009,g2_2010,g2_2011,g2_2012,g2_2013,g2_2014,g2_2015,g2_2016
0,3.656185e-03,3.699090e-03,3.741996e-03,3.784901e-03,3.827806e-03,3.870711e-03,3.913616e-03,3.956521e-03,3.999426e-03,4.042331e-03,...,0.004614,0.003077,0.001540,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
1,4.463398e-03,4.514719e-03,4.566040e-03,4.617361e-03,4.668682e-03,4.720002e-03,4.771323e-03,4.822644e-03,4.873965e-03,4.925286e-03,...,0.005875,0.003934,0.001993,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052
2,4.357183e-03,4.407828e-03,4.458474e-03,4.509120e-03,4.559766e-03,4.610411e-03,4.661057e-03,4.711703e-03,4.762349e-03,4.812994e-03,...,0.005821,0.003895,0.001969,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043
3,4.479640e-03,4.531818e-03,4.583995e-03,4.636173e-03,4.688350e-03,4.740528e-03,4.792705e-03,4.844883e-03,4.897061e-03,4.949238e-03,...,0.005907,0.003955,0.002002,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050
4,3.656185e-03,3.699090e-03,3.741996e-03,3.784901e-03,3.827806e-03,3.870711e-03,3.913616e-03,3.956521e-03,3.999426e-03,4.042331e-03,...,0.004429,0.002953,0.001476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19058,1.546435e-06,1.572332e-06,1.598228e-06,1.624125e-06,1.650022e-06,1.675919e-06,1.701816e-06,1.727712e-06,1.753609e-06,1.779506e-06,...,0.000007,0.000007,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
19059,1.563651e-06,1.605588e-06,1.647525e-06,1.689462e-06,1.731399e-06,1.773336e-06,1.815273e-06,1.857210e-06,1.899147e-06,1.941084e-06,...,0.000007,0.000008,0.000008,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
19060,2.206266e-06,2.281592e-06,2.356917e-06,2.432242e-06,2.507568e-06,2.582893e-06,2.658219e-06,2.733544e-06,2.808870e-06,2.884195e-06,...,0.000012,0.000013,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
19061,5.183284e-07,5.358447e-07,5.533611e-07,5.708774e-07,5.883937e-07,6.059100e-07,6.234263e-07,6.409427e-07,6.584590e-07,6.759753e-07,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003


In [22]:
gdp

,g2_1980,g2_1981,g2_1982,g2_1983,g2_1984,g2_1985,g2_1986,g2_1987,g2_1988,g2_1989,...,g2_2010,g2_2011,g2_2012,g2_2013,g2_2014,g2_2015,g2_2016,lon,lat,geometry
id,,,,,,,,,,,,,,,,,,,,,
340,1.115536e-02,1.124237e-02,1.132938e-02,1.141640e-02,1.150341e-02,1.159042e-02,1.167744e-02,1.176445e-02,1.185146e-02,1.193848e-02,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,-179.861740,-19.004300,POINT (-179.50000 -19.50000)
341,1.371031e-02,1.381203e-02,1.391375e-02,1.401546e-02,1.411718e-02,1.421889e-02,1.432061e-02,1.442232e-02,1.452404e-02,1.462576e-02,...,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052,0.000052,-179.528190,-18.632700,POINT (-179.50000 -18.50000)
342,1.370500e-02,1.380130e-02,1.389760e-02,1.399390e-02,1.409019e-02,1.418649e-02,1.428279e-02,1.437909e-02,1.447539e-02,1.457169e-02,...,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,-179.298957,-17.479223,POINT (-179.50000 -17.50000)
343,1.376388e-02,1.386793e-02,1.397199e-02,1.407605e-02,1.418011e-02,1.428416e-02,1.438822e-02,1.449228e-02,1.459634e-02,1.470039e-02,...,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,-179.673520,-16.418920,POINT (-179.50000 -16.50000)
344,1.058984e-02,1.068515e-02,1.078047e-02,1.087578e-02,1.097109e-02,1.106641e-02,1.116172e-02,1.125704e-02,1.135235e-02,1.144767e-02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-179.702275,-15.829730,POINT (-179.50000 -15.50000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258907,3.538657e-07,3.874342e-07,4.210027e-07,4.545711e-07,4.881396e-07,5.217081e-07,5.552766e-07,5.888451e-07,6.224135e-07,6.559820e-07,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,179.500000,67.500000,POINT (179.50000 67.50000)
258908,3.722025e-07,4.130881e-07,4.539737e-07,4.948592e-07,5.357448e-07,5.766304e-07,6.175159e-07,6.584015e-07,6.992871e-07,7.401726e-07,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,179.499865,68.499858,POINT (179.50000 68.50000)
258909,5.870202e-07,6.555865e-07,7.241528e-07,7.927191e-07,8.612855e-07,9.298518e-07,9.984181e-07,1.066984e-06,1.135551e-06,1.204117e-06,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,179.437835,69.100405,POINT (179.50000 69.50000)


In [23]:
population

,p2_1980,p2_1981,p2_1982,p2_1983,p2_1984,p2_1985,p2_1986,p2_1987,p2_1988,p2_1989,...,p2_2010,p2_2011,p2_2012,p2_2013,p2_2014,p2_2015,p2_2016,lon,lat,geometry
id,,,,,,,,,,,,,,,,,,,,,
340,3.656185e-03,3.699090e-03,3.741996e-03,3.784901e-03,3.827806e-03,3.870711e-03,3.913616e-03,3.956521e-03,3.999426e-03,4.042331e-03,...,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,7.017828e-07,-179.861740,-19.004300,POINT (-179.50000 -19.50000)
341,4.463398e-03,4.514719e-03,4.566040e-03,4.617361e-03,4.668682e-03,4.720002e-03,4.771323e-03,4.822644e-03,4.873965e-03,4.925286e-03,...,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,1.911859e-04,-179.528190,-18.632700,POINT (-179.50000 -18.50000)
342,4.357183e-03,4.407828e-03,4.458474e-03,4.509120e-03,4.559766e-03,4.610411e-03,4.661057e-03,4.711703e-03,4.762349e-03,4.812994e-03,...,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,1.659954e-04,-179.298957,-17.479223,POINT (-179.50000 -17.50000)
343,4.479640e-03,4.531818e-03,4.583995e-03,4.636173e-03,4.688350e-03,4.740528e-03,4.792705e-03,4.844883e-03,4.897061e-03,4.949238e-03,...,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,1.949389e-04,-179.673520,-16.418920,POINT (-179.50000 -16.50000)
344,3.656185e-03,3.699090e-03,3.741996e-03,3.784901e-03,3.827806e-03,3.870711e-03,3.913616e-03,3.956521e-03,3.999426e-03,4.042331e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-179.702275,-15.829730,POINT (-179.50000 -15.50000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258907,1.546435e-06,1.572332e-06,1.598228e-06,1.624125e-06,1.650022e-06,1.675919e-06,1.701816e-06,1.727712e-06,1.753609e-06,1.779506e-06,...,2.675378e-06,2.675378e-06,2.675378e-06,2.675378e-06,2.675378e-06,2.675378e-06,2.675378e-06,179.500000,67.500000,POINT (179.50000 67.50000)
258908,1.563651e-06,1.605588e-06,1.647525e-06,1.689462e-06,1.731399e-06,1.773336e-06,1.815273e-06,1.857210e-06,1.899147e-06,1.941084e-06,...,2.810375e-06,2.810375e-06,2.810375e-06,2.810375e-06,2.810375e-06,2.810375e-06,2.810375e-06,179.499865,68.499858,POINT (179.50000 68.50000)
258909,2.206266e-06,2.281592e-06,2.356917e-06,2.432242e-06,2.507568e-06,2.582893e-06,2.658219e-06,2.733544e-06,2.808870e-06,2.884195e-06,...,4.073253e-06,4.073253e-06,4.073253e-06,4.073253e-06,4.073253e-06,4.073253e-06,4.073253e-06,179.437835,69.100405,POINT (179.50000 69.50000)
